In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


In [3]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [4]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [5]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/AirQualityUCI.csv', sep=';', decimal=',')
df = df.drop(labels=['Date','Time','Unnamed: 15', 'Unnamed: 16'], axis=1)
df.dropna(inplace=True)
#data = clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(df, 1, 1)
data.head()

,CO(GT)(t-1),PT08.S1(CO)(t-1),NMHC(GT)(t-1),C6H6(GT)(t-1),PT08.S2(NMHC)(t-1),NOx(GT)(t-1),PT08.S3(NOx)(t-1),NO2(GT)(t-1),PT08.S4(NO2)(t-1),PT08.S5(O3)(t-1),...,C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
1,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,...,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,...,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,...,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,...,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,...,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848


In [8]:
# x,y=data.loc[:,:],data.loc[:,:]
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# # Get the number of inputs and outputs from the dataset
# df_trainx=x_train.loc[:,'CO(GT)(t-1)':'AH(t-1)']
# df_testx=x_test.loc[:,'CO(GT)(t-1)':'AH(t-1)']
# df_trainy=y_train.loc[:,'CO(GT)(t)':'AH(t)']
# df_original=y_test.loc[:,'CO(GT)(t)':'AH(t)']

# n_inputs, n_outputs = x.shape[1], y.shape[1]

In [9]:
# MultiOutputRegressor(estimator=SVR(C=45.6101792821259,
#                                    epsilon=0.9795728606357624))

# MultiOutputRegressor(estimator=SVR(C=9.984235532108787,
#                                    epsilon=0.00019155042003382583))

def create_and_fit_model(X_train,y_train):
    clf = MultiOutputRegressor(estimator=SVR(C=9.984235532108787, epsilon=0.00019155042003382583))
    clf.fit(X_train, y_train)
    return clf

In [10]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [11]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            
#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            Xtrain = scaler.fit_transform(ttrain.loc[:,'CO(GT)(t-1)':'AH(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'CO(GT)(t)':'AH(t)'])
            Xtest = scaler.transform(ttest.loc[:,'CO(GT)(t-1)':'AH(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'CO(GT)(t)':'AH(t)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ttest.loc[:,'CO(GT)(t)':'AH(t)'].columns
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'CO(GT)(t)':'AH(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [12]:
n_windows = 30
train_size = 0.75 

model_svr = sliding_window(data,n_windows,train_size)

--------------------
training window 0
--------------------
training window 311
--------------------
training window 622
--------------------
training window 933
--------------------
training window 1244


<ipython-input-4-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 1555


<ipython-input-4-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 1866


...

<ipython-input-4-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 8708


<ipython-input-4-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


--------------------
training window 9019


<ipython-input-4-1be783193a86>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


In [13]:
df_forecats_columns = data.loc[:,'CO(GT)(t)':'AH(t)'].columns

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": []
}

measures = model_svr
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [15]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse
0,CO(GT)(t),36.944,16.729,372.644,0.111,41.420,inf
1,PT08.S1(CO)(t),185.162,130.001,25.277,0.098,7.545,0.185
2,NMHC(GT)(t),30.527,22.205,9.581,0.896,4.286,NaN
3,C6H6(GT)(t),17.339,9.431,60.398,-0.033,23.720,0.202
4,PT08.S2(NMHC)(t),208.455,151.671,28.303,0.279,9.567,0.172
5,NOx(GT)(t),122.004,86.857,48.780,0.271,23.502,inf
6,PT08.S3(NOx)(t),175.617,127.808,27.852,0.381,8.941,0.161
7,NO2(GT)(t),74.803,44.391,31.192,0.183,18.214,inf
8,PT08.S4(NO2)(t),251.424,172.223,32.485,0.093,7.479,0.179
9,PT08.S5(O3)(t),268.277,207.903,35.835,0.127,12.441,0.180


In [16]:
final_measures.to_csv (r'svr_uci_air_quality_italy.csv', index = False, header=True)

## RandomizedSearchCV

In [7]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=10)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'estimator__C': 45.6101792821259, 'estimator__epsilon': 0.9795728606357624, 'estimator__kernel': 'rbf'}


In [8]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=SVR(C=45.6101792821259,
                                   epsilon=0.9795728606357624))

In [9]:
y_p=tuned_model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,2.177120,1094.458438,-200.192685,8.984933,938.024896,215.580605,769.541654,118.775808,1506.517886,1002.912784,23.950856,48.358112,1.414877
1,-196.998117,886.817525,-200.258057,4.036219,729.020041,26.235476,891.994569,29.567728,1489.469804,767.425425,24.201814,54.641173,0.980556
2,1.874302,1068.741630,-200.124681,8.179867,919.765280,389.431874,700.791164,175.308945,1099.044186,1082.003752,8.990329,48.319495,0.509350
3,2.315464,1221.863499,-200.178447,7.735720,886.928247,309.778935,708.385258,132.786204,1319.639872,1045.679597,13.696309,70.207073,1.446077
4,-0.287457,871.988821,-199.782807,3.546551,626.543765,118.672605,1098.103255,81.274518,882.902163,732.815269,9.056460,43.879590,1.101661
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,3.011671,1021.543653,-200.034978,10.572348,989.926200,363.339874,778.056422,142.540428,1197.907494,1021.936559,15.285857,39.776187,0.973641
2335,-194.662074,778.819127,-200.732543,2.074473,558.774293,-181.439576,1209.267996,-183.806292,1162.865054,560.562391,13.086992,64.583396,0.721327
2336,5.127206,-132.054824,-199.333059,-198.025759,-150.703817,891.742488,-158.475147,215.176905,-107.797755,-127.812903,-198.009899,-197.698377,-198.159215
2337,-0.446139,868.168961,-199.763517,2.400820,609.956105,40.074804,1109.896595,34.907182,1335.846426,533.480770,16.668726,70.823198,1.420227


In [10]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [12]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('CO(GT)(t)', 39.72, 9.943, 0.729, 10.63)
Results: ('PT08.S1(CO)(t)', 126.746, 73.53, 0.859, 0.073)
Results: ('NMHC(GT)(t)', 51.853, 11.322, 0.888, 0.052)
Results: ('C6H6(GT)(t)', 12.216, 3.157, 0.916, 0.314)
Results: ('PT08.S2(NMHC)(t)', 146.135, 93.863, 0.827, 0.104)
Results: ('NOx(GT)(t)', 118.67, 69.932, 0.791, 0.459)
Results: ('PT08.S3(NOx)(t)', 120.082, 76.988, 0.866, 0.103)
Results: ('NO2(GT)(t)', 70.963, 32.145, 0.686, 0.331)
Results: ('PT08.S4(NO2)(t)', 168.882, 96.211, 0.874, 0.076)
Results: ('PT08.S5(O3)(t)', 186.305, 125.463, 0.84, 0.136)
Results: ('T(t)', 12.107, 2.313, 0.923, 0.137)
Results: ('RH(t)', 14.705, 4.747, 0.919, 0.104)
Results: ('AH(t)', 11.021, 0.96, 0.922, 0.72)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,CO(GT)(t),39.720,9.943,0.729,10.630
1,PT08.S1(CO)(t),126.746,73.530,0.859,0.073
2,NMHC(GT)(t),51.853,11.322,0.888,0.052
3,C6H6(GT)(t),12.216,3.157,0.916,0.314
4,PT08.S2(NMHC)(t),146.135,93.863,0.827,0.104
5,NOx(GT)(t),118.670,69.932,0.791,0.459
6,PT08.S3(NOx)(t),120.082,76.988,0.866,0.103
7,NO2(GT)(t),70.963,32.145,0.686,0.331
8,PT08.S4(NO2)(t),168.882,96.211,0.874,0.076
9,PT08.S5(O3)(t),186.305,125.463,0.840,0.136


## MinMaxScaler

In [13]:
#data_red = data.loc[:10000,:]
x,y=data.loc[:,:],data.loc[:,:]

In [14]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery=MinMaxScaler()

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'CO(GT)(t-1)':'AH(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'CO(GT)(t-1)':'AH(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'CO(GT)(t)':'AH(t)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'CO(GT)(t)':'AH(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [16]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=10)

hyperparameters_tuning_nor = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning_nor.best_estimator_

Best Parameters = {'estimator__C': 45.6101792821259, 'estimator__epsilon': 0.9795728606357624, 'estimator__kernel': 'rbf'}


In [17]:
hyperparameters_tuning_nor.best_estimator_

MultiOutputRegressor(estimator=SVR(C=9.984235532108787,
                                   epsilon=0.00019155042003382583))

In [18]:

# x,y=data.loc[:,:],data.loc[:,:]

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# # Get the number of inputs and outputs from the dataset
# df_trainx=x_train.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
# df_trainx_scaled = scalerx.fit_transform(df_trainx)
# df_testx=x_test.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
# df_testx_scaled = scalerx.transform(df_testx)
# df_trainy=y_train.loc[:,'Appliances(t)':'Tdewpoint(t)']
# df_trainy_scaled=scalery.fit_transform(df_trainy)

# df_original=y_test.loc[:,'Appliances(t)':'Tdewpoint(t)']

In [19]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,0.959423,0.567660,-0.000255,0.788980,0.457457,0.233489,0.343515,0.569945,0.565740,0.418223,0.918503,0.858727,0.996056
1,0.086146,0.486076,0.000063,0.774442,0.382617,0.157131,0.373415,0.472305,0.572566,0.366721,0.910747,0.879551,0.996308
2,0.961274,0.557208,0.000045,0.788028,0.456697,0.332420,0.320269,0.688083,0.429881,0.459043,0.847148,0.860774,0.991593
3,0.961762,0.638861,-0.000022,0.786368,0.446125,0.308750,0.310717,0.607356,0.513244,0.459025,0.873296,0.936150,0.994553
4,0.955277,0.478530,0.000038,0.769223,0.343388,0.189607,0.447077,0.515922,0.367231,0.350545,0.857458,0.836849,0.991758
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,0.963725,0.522661,0.000034,0.791818,0.474731,0.322124,0.355401,0.610148,0.454115,0.426877,0.874642,0.835397,0.992344
2335,0.000129,0.435044,0.000098,0.766375,0.321321,-0.000229,0.487054,-0.000098,0.462324,0.292009,0.873035,0.933677,0.994587
2336,0.993160,0.000436,-0.000096,0.000393,0.000339,0.800401,0.000535,0.765345,0.000278,0.000721,0.000226,0.000447,0.000424
2337,0.953629,0.475862,0.000086,0.767071,0.332218,0.147987,0.438423,0.438928,0.520044,0.275525,0.891199,0.966223,0.996947


In [20]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
0,1.670622,1071.558853,-200.354261,8.054047,904.301209,192.028514,790.354245,107.770083,1483.077439,938.821278,24.665719,46.540589,1.379129
1,-181.892007,888.809183,-199.912724,4.220345,723.636732,63.823078,876.556404,55.044954,1503.384965,798.582433,22.768752,52.519167,1.430128
2,2.059697,1048.146950,-199.937032,7.802967,902.466811,358.133589,723.336780,171.564681,1078.894938,1049.974131,7.212301,47.128142,0.476853
3,2.162365,1231.048289,-200.030446,7.365235,876.944590,318.392010,695.797422,127.972143,1326.900166,1049.926096,13.608296,68.768698,1.075365
4,0.799302,871.908022,-199.946767,2.844189,628.938337,118.350133,1088.921865,78.598064,892.510888,754.534620,9.734115,40.259275,0.510291
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2334,2.574901,970.759709,-199.953029,8.802483,946.000534,340.846263,824.621785,129.480032,1150.991823,962.386180,13.937500,39.842388,0.628806
2335,-199.972860,774.497803,-199.863526,2.093160,575.669438,-200.384780,1204.176993,-200.052961,1175.413389,595.139324,13.544352,68.058723,1.082316
2336,8.762229,-199.023976,-200.132840,-199.896344,-199.182693,1143.873386,-198.458791,213.286395,-199.173761,-198.035604,-199.944626,-199.871751,-199.914352
2337,0.452838,865.930415,-199.879998,2.276647,601.973934,48.469868,1063.974490,37.021077,1347.131425,550.255729,17.987287,77.402765,1.559328


In [21]:
df_original

,CO(GT)(t),PT08.S1(CO)(t),NMHC(GT)(t),C6H6(GT)(t),PT08.S2(NMHC)(t),NOx(GT)(t),PT08.S3(NOx)(t),NO2(GT)(t),PT08.S4(NO2)(t),PT08.S5(O3)(t),T(t),RH(t),AH(t)
4636,1.7,1077.0,-200.0,8.5,920.0,169.0,778.0,99.0,1509.0,935.0,23.6,50.0,1.4383
3396,-200.0,894.0,-200.0,4.9,761.0,36.0,862.0,48.0,1504.0,768.0,23.0,49.9,1.3849
8632,1.1,915.0,-200.0,3.1,658.0,166.0,957.0,121.0,866.0,562.0,10.0,37.7,0.4643
6984,2.3,1286.0,-200.0,9.1,942.0,343.0,687.0,137.0,1362.0,1103.0,13.5,67.1,1.0357
6612,1.0,909.0,-200.0,4.3,728.0,221.0,933.0,107.0,948.0,905.0,7.8,47.4,0.5046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,3.1,1064.0,-200.0,12.2,1055.0,412.0,738.0,138.0,1242.0,1063.0,14.1,43.0,0.6883
5314,-200.0,777.0,-200.0,1.8,570.0,-200.0,1191.0,-200.0,1167.0,616.0,14.5,65.0,1.0672
6747,8.6,-200.0,-200.0,-200.0,-200.0,1237.0,-200.0,203.0,-200.0,-200.0,-200.0,-200.0,-200.0000
5147,0.6,903.0,-200.0,3.4,679.0,90.0,956.0,44.0,1385.0,660.0,18.2,76.9,1.5945


In [22]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)
 

In [23]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('CO(GT)(t)', 35.57, 7.896, 0.783, 6.642)
Results: ('PT08.S1(CO)(t)', 120.895, 69.703, 0.872, 0.069)
Results: ('NMHC(GT)(t)', 41.89, 8.822, 0.927, 0.041)
Results: ('C6H6(GT)(t)', 12.152, 3.033, 0.916, 0.292)
Results: ('PT08.S2(NMHC)(t)', 139.256, 89.282, 0.843, 0.099)
Results: ('NOx(GT)(t)', 107.42, 59.774, 0.829, 0.319)
Results: ('PT08.S3(NOx)(t)', 112.705, 72.806, 0.882, 0.098)
Results: ('NO2(GT)(t)', 62.662, 25.479, 0.755, 0.218)
Results: ('PT08.S4(NO2)(t)', 161.314, 91.955, 0.885, 0.072)
Results: ('PT08.S5(O3)(t)', 177.17, 118.696, 0.855, 0.128)
Results: ('T(t)', 11.922, 1.504, 0.926, 0.079)
Results: ('RH(t)', 14.243, 3.619, 0.924, 0.076)
Results: ('AH(t)', 10.993, 0.642, 0.922, 0.362)
Statistics MIMO (test): 


,variable,rmse,mae,r2,mape
0,CO(GT)(t),35.570,7.896,0.783,6.642
1,PT08.S1(CO)(t),120.895,69.703,0.872,0.069
2,NMHC(GT)(t),41.890,8.822,0.927,0.041
3,C6H6(GT)(t),12.152,3.033,0.916,0.292
4,PT08.S2(NMHC)(t),139.256,89.282,0.843,0.099
5,NOx(GT)(t),107.420,59.774,0.829,0.319
6,PT08.S3(NOx)(t),112.705,72.806,0.882,0.098
7,NO2(GT)(t),62.662,25.479,0.755,0.218
8,PT08.S4(NO2)(t),161.314,91.955,0.885,0.072
9,PT08.S5(O3)(t),177.170,118.696,0.855,0.128
